In [1]:
import math
from itertools import combinations
import sys


def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])


def circumcircle_radius(p1, p2, p3):
    a = distance(p2, p3)
    b = distance(p1, p3)
    c = distance(p1, p2)
    s = (a + b + c) / 2
    area_sq = s * (s - a) * (s - b) * (s - c)
    if area_sq <= 0:
        return None
    area = math.sqrt(area_sq)
    radius = (a * b * c) / (4 * area)
    return radius


def possible_circle_centers(p1, p2, R):
    mid_x = (p1[0] + p2[0]) / 2
    mid_y = (p1[1] + p2[1]) / 2
    d = distance(p1, p2)
    if d > 2 * R:
        return []
    if d == 0:
        return []
    h = math.sqrt(R**2 - (d / 2)**2)
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    norm = math.hypot(dx, dy)
    if norm == 0:
        return []
    dx /= norm
    dy /= norm
    perp1 = (-dy, dx)
    perp2 = (dy, -dx)
    center1 = (mid_x + perp1[0] * h, mid_y + perp1[1] * h)
    center2 = (mid_x + perp2[0] * h, mid_y + perp2[1] * h)
    return [center1, center2]


def read_datasets(filepath):
    datasets = []
    with open(filepath, 'r') as file:
        for line in file:
            numbers = list(map(float, line.strip().split()))
            if len(numbers) != 72:
                continue
            points = [(numbers[i], numbers[i+1]) for i in range(0, 72, 2)]
            datasets.append(points)
            if len(datasets) == 10:
                break
    return datasets


def exists_circle_covering_k_points(R, points, k, epsilon=1e-9):
    n = len(points)
    for p1, p2 in combinations(points, 2):
        if distance(p1, p2) > 2 * R + epsilon:
            continue
        centers = possible_circle_centers(p1, p2, R)
        for center in centers:
            count = 0
            for p in points:
                if distance(center, p) <= R + epsilon:
                    count += 1
                if count >= k:
                    return True
    for p in points:
        count = 0
        for q in points:
            if distance(p, q) <= R + epsilon:
                count += 1
            if count >= k:
                return True
    return False


def find_min_radius(points, k=6):
    candidate_radii = set()
    for p1, p2 in combinations(points, 2):
        d = distance(p1, p2)
        candidate_radii.add(d / 2)
    for p1, p2, p3 in combinations(points, 3):
        R = circumcircle_radius(p1, p2, p3)
        if R is not None:
            candidate_radii.add(R)
    for p in points:
        sorted_distances = sorted([distance(p, q) for q in points])
        if len(sorted_distances) >= k:
            candidate_radii.add(sorted_distances[k-1])
    sorted_radii = sorted(candidate_radii)
    left = 0
    right = len(sorted_radii) - 1
    minimal_R = sorted_radii[-1]
    while left <= right:
        mid = (left + right) // 2
        R = sorted_radii[mid]
        if exists_circle_covering_k_points(R, points, k):
            minimal_R = R
            right = mid - 1
        else:
            left = mid + 1
    return minimal_R


def main():
    filepath = 'data/536_6_of_36/data.txt'
    datasets = read_datasets(filepath)
    results = []
    for idx, points in enumerate(datasets):
        min_R = find_min_radius(points, k=6)
        results.append("{0:.15f}".format(min_R))
    for res in results:
        print(res)


if __name__ == "__main__":
    main()

5.493332994971635
5.797937478778344
4.784397459401070
4.837354835678288
6.235095992997768
4.806198386726145
6.377093710120439
4.129550769168280
6.811447926504941
6.380491896557452
